In [95]:
import requests
import pandas as pd

def download_raw_file():
    url = "https://ir.eia.gov/wpsr/psw09.xls"
    response = requests.get(url)
    file_path = "./data/eia_weekly_psw09.xls"
    with open(file_path, "wb") as file:
        file.write(response.content)

def read_excel_file():
    file_path = './data/eia_weekly_psw09.xls'
    sheets = pd.read_excel(file_path, sheet_name=None)
    contents = sheets.pop('Contents')
    return sheets

def parse_data(df):
    df = df.copy()
    df.columns = df.iloc[0] + '___' + df.iloc[1]
    df = df.drop([0, 1])
    df = df.rename(columns={df.columns[0]: 'period'})
    df['period'] = pd.to_datetime(df['period'])
    df = df[df['period'] >= '2024-01-01']
    df = df.melt(id_vars=['period'], var_name='id', value_name='value')
    df[['id', 'name']] = df['id'].str.split('___', expand=True)
    return df

def parse_all_data(sheets):
    df = pd.DataFrame()
    for sheet in sheets:
        df = pd.concat([df, parse_data(sheets[sheet])])
    df = df.drop_duplicates(subset=['period', 'id'])
    df = df[['period','id','name','value']]
    
    return df

def last_two_weeks(df):
    df = df.copy()
    max_date = df['period'].max()
    prior_week = max_date - pd.DateOffset(weeks=1)
    df = df[(df['period'] == max_date) | (df['period'] == prior_week)]
    df = df.dropna(subset=['value'])
    return df

def pivot_data(df):
    df = df.copy()
    df['period'] = df['period'].dt.strftime('%m/%d')
    df = df.pivot(index='period', columns=['id','name'], values='value').T.reset_index()
    df.columns.name = None
    return df

def main():
    download_raw_file()
    sheets = read_excel_file()
    df = parse_all_data(sheets)
    df = last_two_weeks(df)
    df = pivot_data(df)
    return df

if __name__ == "__main__":
    df = main()
    df.to_csv('xlsx_download.csv', index=False)


In [96]:
production_mapping = {
    
    #Crude Stocks
    'WCESTUS1' : 'US Commercial Stocks (kb)',
    'WCESTP11' : 'P1 Commercial Stocks (kb)',
    'WCESTP21' : 'P2 Commercial Stocks (kb)',
    'WCESTP31' : 'P3 Commercial Stocks (kb)',
    'WCESTP41' : 'P4 Commercial Stocks (kb)',
    'WCESTP51' : 'P5 Commercial Stocks (kb)',
    #Other Crude Stocks
    'W_EPC0_SAX_YCUOK_MBBL' : 'Cushing Stocks (kb)',
    'crudeStocksP2E' : 'P2E Stocks (kb)',
    'WCSSTUS1' : 'SPR Stocks (kb)',
    'W_EPC0_SKA_NUS_MBBL' : 'Alaska Stocks (kb)',
    'WCRSTUS1' : 'Total Stocks (kb)',
    #Crude Exports
    'WCREXUS2' : 'US Crude Exports (kbd)',
    #Crude Runs
    'WCRRIUS2' : 'US Refinery Runs (kbd)',
    'WCRRIP12' : 'P1 Refinery Runs (kbd)',
    'WCRRIP22' : 'P2 Refinery Runs (kbd)',
    'WCRRIP32' : 'P3 Refinery Runs (kbd)',
    'WCRRIP42' : 'P4 Refinery Runs (kbd)',
    'WCRRIP52' : 'P5 Refinery Runs (kbd)',
    #Crude Imports
    'WCEIMUS2' : 'US Imports (kbd)',
    'WCEIMP12' : 'P1 Imports (kbd)',
    'WCEIMP22' : 'P2 Imports (kbd)',
    'WCEIMP32' : 'P3 Imports (kbd)',
    'WCEIMP42' : 'P4 Imports (kbd)',
    'WCEIMP52' : 'P5 Imports (kbd)',
    #Crude Production
    'WCRFPUS2': 'US Production (kbd)',
    'W_EPC0_FPF_R48_MBBLD': 'L48 Production (kbd)',
    'W_EPC0_FPF_SAK_MBBLD': 'AK Production (kbd)',        
    #Crude Adjustments
    'crudeOriginalAdjustment' : 'OG Adjustment Factor (kbd)',
    # 'W_EPC0_TVP_NUS_MBBLD' : 'US Transfer to Crude Supply of Crude (kbd)',
    # 'W_EPLLCL2A_TVP_NUS_MBBLD' : 'US Trf to Crude Supply of Condy. Prod. and AK NGL (kbd)',
    # 'W_EPLLNGPU_TVP_NUS_MBBLD' : 'US Trf to Crude Supply of NG and Unf. Oil Prod. Spld. (kbd)',
    # 'WCRAUUS2' : 'US Crude Adjustment Factor (kbd)',
    #Crude Refinery Utilizaiton
    'WPULEUS3' : 'US Refinery Utilization (pct)',
    'W_NA_YUP_R10_PER' : 'P1 Refinery Utilization (pct)',
    'W_NA_YUP_R20_PER' : 'P2 Refinery Utilization (pct)',
    'W_NA_YUP_R30_PER' : 'P3 Refinery Utilization (pct)',
    'W_NA_YUP_R40_PER' : 'P4 Refinery Utilization (pct)',
    'W_NA_YUP_R50_PER' : 'P5 Refinery Utilization (pct)',
    #Feedstock Runs
    'feedstockRunsUS' : 'US Feedstock Runs (kbd)',
    'feddStockRunsP1' : 'P1 Feedstock Runs (kbd)',
    'feedstockRunsP2' : 'P2 Feedstock Runs (kbd)',
    'feedstockRunsP3' : 'P3 Feedstock Runs (kbd)',
    'feedstockRunsP4' : 'P4 Feedstock Runs (kbd)',
    'feedstockRunsP5' : 'P5 Feedstock Runs (kbd)',
    #Gross Runs
    'WGIRIUS2' : 'US Gross Runs (kbd)',
    'WGIRIP12' : 'P1 Gross Runs (kbd)',
    'WGIRIP22' : 'P2 Gross Runs (kbd)',
    'WGIRIP32' : 'P3 Gross Runs (kbd)',
    'WGIRIP42' : 'P4 Gross Runs (kbd)',
    'WGIRIP52' : 'P5 Gross Runs (kbd)',        
    #PADD9 Stats
    'crudeStocksP9' : 'P9 Stocks (kb)',
    'crudeRunsP9' : 'P9 Crude Runs (kbd)',
    'grossRunsP9' : 'P9 Gross Runs (kbd)',
    'feedstockRunsP9' : 'P9 Feedstock Runs (kbd)',
    'crudeImportsP9' : 'P9 Crude Imports (kbd)',
    #Crude Country Level Imports
    'W_EPC0_IM0_NUS-NBR_MBBLD' : 'Brazil Imports (kbd)',
    'W_EPC0_IM0_NUS-NCA_MBBLD' : 'Canada Imports (kbd)',
    'W_EPC0_IM0_NUS-NCO_MBBLD' : 'Colombia Imports (kbd)',
    'W_EPC0_IM0_NUS-NEC_MBBLD' : 'Ecuador Imports (kbd)',
    'W_EPC0_IM0_NUS-NIZ_MBBLD' : 'Iraq Imports (kbd)',
    'W_EPC0_IM0_NUS-NMX_MBBLD' : 'Mexico Imports (kbd)',
    'W_EPC0_IM0_NUS-NNI_MBBLD' : 'Nigeria Imports (kbd)',
    'W_EPC0_IM0_NUS-NRS_MBBLD' : 'Russia Imports (kbd)',
    'W_EPC0_IM0_NUS-NSA_MBBLD' : 'Saudi Arabia Imports (kbd)',    
    
    #Gasoline Stocks
    'WGTSTUS1' : 'USGasoline Stocks (kb)',
    'WGTSTP11' : 'P1 Gasoline Stocks (kb)',
    'WGTSTP21' : 'P2 Gasoline Stocks (kb)',
    'WGTSTP31' : 'P3 Gasoline Stocks (kb)',
    'WGTSTP41' : 'P4 Gasoline Stocks (kb)',
    'WGTSTP51' : 'P5 Gasoline Stocks (kb)',
    #Gasoline Imports
    'WGTIMUS2' : 'US Gasoline Imports (kbd)',
    'WGTIM_R10-Z00_2' : 'P1 Gasoline Imports (kbd)',
    'WGTIM_R20-Z00_2' : 'P2 Gasoline Imports (kbd)',
    'WGTIM_R30-Z00_2' : 'P3 Gasoline Imports (kbd)',
    'WGTIM_R40-Z00_2' : 'P4 Gasoline Imports (kbd)',
    'WGTIM_R50-Z00_2' : 'P5 Gasoline Imports (kbd)',
    #Gasoline Production
    'WGFRPUS2' : 'US Gasoline Production (kbd)',
    'WGFRPP12' : 'P1 Gasoline Production (kbd)',
    'WGFRPP22' : 'P2 Gasoline Production (kbd)',
    'WGFRPP32' : 'P3 Gasoline Production (kbd)',
    'WGFRPP42' : 'P4 Gasoline Production (kbd)',
    'WGFRPP52' : 'P5 Gasoline Production (kbd)',
    #Gasoline Exports
    'W_EPM0F_EEX_NUS-Z00_MBBLD' : 'US Gasoline Exports (kbd)',

    #Distillate Stocks
    'WDISTUS1' : 'US Distillate Stocks (kb)',
    'WDISTP11' : 'P1 Distillate Stocks (kb)',
    'WDISTP21' : 'P2 Distillate Stocks (kb)',
    'WDISTP31' : 'P3 Distillate Stocks (kb)',
    'WDISTP41' : 'P4 Distillate Stocks (kb)',
    'WDISTP51' : 'P5 Distillate Stocks (kb)',
    #Distillate Imports
    'WDIIMUS2' : 'US Distillate Imports (kbd)',
    'WDIIM_R10-Z00_2' : 'P1 Distillate Imports (kbd)',
    'WDIIM_R20-Z00_2' : 'P2 Distillate Imports (kbd)',
    'WDIIM_R30-Z00_2' : 'P3 Distillate Imports (kbd)',
    'WDIIM_R40-Z00_2' : 'P4 Distillate Imports (kbd)',
    'WDIIM_R50-Z00_2' : 'P5 Distillate Imports (kbd)',
    #Distillate Produciton
    'WDIRPUS2' : 'US Distillate Production (kbd)',
    'WDIRPP12' : 'P1 Distillate Production (kbd)',
    'WDIRPP22' : 'P2 Distillate Production (kbd)',
    'WDIRPP32' : 'P3 Distillate Production (kbd)',
    'WDIRPP42' : 'P4 Distillate Production (kbd)',
    'WDIRPP52' : 'P5 Distillate Production (kbd)',
    #Distillate Exports
    'WDIEXUS2' : 'US Distillate Exports (kbd)',

    #Jet Stocks
    'WKJSTUS1' : 'US Jet Stocks (kb)',
    'WKJSTP11' : 'P1 Jet Stocks (kb)',
    'WKJSTP21' : 'P2 Jet Stocks (kb)',
    'WKJSTP31' : 'P3 Jet Stocks (kb)',
    'WKJSTP41' : 'P4 Jet Stocks (kb)',
    'WKJSTP51' : 'P5 Jet Stocks (kb)',
    #Jet Imports
    'WKJIMUS2' : 'US Jet Imports (kbd)',
    'WKJIM_R10-Z00_2' : 'P1 Jet Imports (kbd)',
    'WKJIM_R20-Z00_2' : 'P2 Jet Imports (kbd)',
    'WKJIM_R30-Z00_2' : 'P3 Jet Imports (kbd)',
    'WKJIM_R40-Z00_2' : 'P4 Jet Imports (kbd)',
    'WKJIM_R50-Z00_2' : 'P5 Jet Imports (kbd)',
    #Jet Production
    'WKJRPUS2' : 'US Jet Production (kbd)',
    'WKJRPP12' : 'P1 JetProduction (kbd)',
    'WKJRPP22' : 'P2 Jet Production (kbd)',
    'WKJRPP32' : 'P3 Jet Production (kbd)',
    'WKJRPP42' : 'P4 Jet Production (kbd)',
    'WKJRPP52' : 'P5 Jet Production (kbd)',
    #Jet Exports
    'WKJEXUS2' : 'US Jet Exports (kbd)',
    
    #Fuel Oil Stocks
    'WRESTUS1' : 'US Fuel Oil Stocks (kb)',
    'WRESTP11' : 'P1 Fuel Oil Stocks (kb)',
    'WRESTP21' : 'P2 Fuel Oil Stocks (kb)',
    'WRESTP31' : 'P3 Fuel Oil Stocks (kb)',
    'WRESTP41' : 'P4 Fuel Oil Stocks (kb)',
    'WRESTP51' : 'P5 Fuel Oil Stocks (kb)',
    #Fuel Oil Imports
    'WREIMUS2' : 'US Fuel Oil Imports (kbd)',
    'WREIM_R10-Z00_2' : 'P1 Fuel Oil Imports (kbd)',
    'WREIM_R20-Z00_2' : 'P2 Fuel Oil Imports (kbd)',
    'WREIM_R30-Z00_2' : 'P3 Fuel Oil Imports (kbd)',
    'WREIM_R40-Z00_2' : 'P4 Fuel Oil Imports (kbd)',
    'WREIM_R50-Z00_2' : 'P5 Fuel Oil Imports (kbd)',
    #Fuel Oil Production
    'WRERPUS2' : 'US Fuel Oil Production (kbd)',
    'WRERPP12' : 'P1 Fuel Oil Production (kbd)',
    'WRERPP22' : 'P2 Fuel Oil Production (kbd)',
    'WRERPP32' : 'P3 Fuel Oil Production (kbd)',
    'WRERPP42' : 'P4 Fuel Oil Production (kbd)',
    'WRERPP52' : 'P5 Fuel Oil Production (kbd)',
    #Fuel Oil Exports
    'WREEXUS2' : 'US Fuel Oil Exports (kbd)',    
    			
    #C3/C3= Stocks
    'WPRSTUS1' : 'US C3/C3= Stocks (kb)',
    'WPRSTP11' : 'P1 C3/C3= Stocks (kb)',
    'WPRSTP21' : 'P2 C3/C3= Stocks (kb)',
    'WPRSTP31' : 'P3 C3/C3= Stocks (kb)',
    'WPRST_R4N5_1' : 'P4P5 C3/C3= Stocks (kb)',
    #C3/C3= Imports
    'WPRIM_NUS-Z00_2' : 'US C3/C3= Imports (kbd)',
    'WPRIMP12' : 'P1 C3/C3= Imports (kbd)',
    'WPRIMP22' : 'P2 C3/C3= Imports (kbd)',
    'WPRIMP32' : 'P3 C3/C3= Imports (kbd)',
    'W_EPLLPZ_IM0_R45-Z00_MBBLD' : 'P4P5 C3/C3= Imports (kbd)',
    #C3/C3= Production
    'WPRTP_NUS_2' : 'US C3/C3= Production (kbd)',
    'WPRNPP12' : 'P1 C3/C3= Production (kbd)',
    'WPRNPP22' : 'P2 C3/C3= Production (kbd)',
    'WPRNPP32' : 'P3 C3/C3= Production (kbd)',
    'W_EPLLPZ_YPT_R4N5_MBBLD' : 'P4P5 C3/C3= Production (kbd)',
    #C3/C3= Exports
    'W_EPLLPZ_EEX_NUS-Z00_MBBLD' : 'US C3/C3= Exports (kbd)',

    #Products Supplied
    'WRPUPUS2' : 'Products Supplied (kbd)',
    'WGFUPUS2' : 'Gasoline Supplied (kbd)',
    'WDIUPUS2' : 'Distillate Supplied (kbd)',
    'WKJUPUS2' : 'Jet Supplied (kbd)',
    'WREUPUS2' : 'Fuel Oil Supplied (kbd)',
    'WPRUP_NUS_2' : ' C3/C3= Supplied (kbd)',
    'WWOUP_NUS_2' : 'Other Oils Supplied (kbd)',
}

df = pd.DataFrame(production_mapping.items(), columns=['id', 'name'])

In [98]:
df.to_csv('production_mapping.csv', index=False)